# Aufgabe5 – Interpretation von Modellen

## Notebook Beschreibung
...

## Bibliotheken importieren und Notebook setup

In [ ]:
# Modul Pfad setzen
import sys
sys.path.append('./module')


# Bibliotheken importieren
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score


## Analyse und Datenvorverarbeitung

Aufgabe 5a)  
Lesen Sie den Datensatz Hdma.csv ein und machen Sie sich mit den Daten vertraut.

In [ ]:
# Load Dataset from csv (use ; as separator)
data = pd.read_csv('./data/Hdma.csv', sep=';')
# Display first line to test if data was loaded correctly
data.head()

In [ ]:
# Show dataframe size
data.shape

Der Datensatz besteht aus 2381 Datenpunkten mit jeweils 13 Merkmalen

In [ ]:
data.isnull().any()

Die Spalten "pbcr" und "self" enthalten fehlende Daten.

In [ ]:
data[data.isnull().any(axis=1)]

Nur in einer Zeile ist "pbcr" und "self" gleich NaN. Da es nur eine einzige Zeile mit fehlenden Daten gibt, haben wir uns entschieden diese aus dem Datensatz zu entfernen.

In [ ]:
data = data.drop(2380)
data.shape

### Art der Merkmale 

In [ ]:
data.columns

- `dir` numerisch-kontinuierlich 
- `hir` numerisch-kontinuierlich
- `lvr` numerisch-kontinuierlich
- `ccs` kategorisch-ordinal
- `mcs` kategorisch-ordinal
- `pbcr` kategorisch-nominal
- `dmi` kategorisch-nominal
- `self` kategorisch-nominal
- `single` kategorisch-nominal
- `uria` numerisch-kontinuierlich
- `condo` kategorisch-nominal
- `black` kategorisch-nominal
- `deny` kategorisch-nominal


Die Zielvariable der Klassifikation ist 'deny' welche angibt ob der Kreditantrag abgelehnt wurde. Es soll bestimmt werden welche Merkmale besonderen Einfluss auf diese haben. Im Speziellen soll bestimmt werden, ob es eine rassistische Diskriminierung gibt. Dafür muss das Merkmal 'black' betrachtet werden

In [ ]:
sns.countplot(x='deny',data=data)
plt.show()

In [ ]:
nominal_features = ['pbcr', 'dmi', 'self', 'single', 'condo', 'black', 'deny']
labelencoder = LabelEncoder()
data[nominal_features] = data[nominal_features].apply(labelencoder.fit_transform)
data.head()

In [ ]:
continuous_features = ['dir', 'hir', 'lvr']
for feature in continuous_features:
    plt.figure(figsize=(14, 4))
    sns.boxplot(data[feature])
    plt.show()

In [ ]:
outliers = data[data["dir"]>2]
outliers.head()

In [ ]:
data.drop(outliers.index, inplace=True)
data.shape

In [ ]:
correlation_matrix = data.corr()
fig = plt.figure(figsize=(12,9))
sns.heatmap(correlation_matrix,vmin=-1,vmax=1,annot=True)
plt.show()

In [ ]:
# Create numpy arrays for features and target
X = data.drop("deny", axis=1).to_numpy()
y = data["deny"].to_numpy()
# Create 80/20 train val split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)
# Print resulting dataset sizes
print("Shape X_train", X_train.shape)
print("Shape y_train", y_train.shape)
print("Shape X_test", X_test.shape)
print("Shape y_test", y_test.shape)

In [ ]:
# Use StandardScaler to transform data to zero mean and unit variance
sc = StandardScaler()
# Fit and transform on training data
X_train = sc.fit_transform(X_train)
# Only apply transformation to test data
X_test = sc.transform(X_test)

In [ ]:
cls = LogisticRegression()
cls.fit(X_train, y_train)
print(f1_score(y_train, cls.predict(X_train)))
print(f1_score(y_test, cls.predict(X_test)))
print(f1_score(y_train, np.zeros_like(y_train)))
print(f1_score(y_test, np.zeros_like(y_test)))


In [ ]:
cls.coef_.squeeze().tolist()

In [ ]:
list(zip(data.columns.to_list(), cls.coef_.squeeze().tolist()))

In [ ]:
data.columns.to_list()